# Optimal Path Algorithms between IDA* and Hill Climbing algo

## Problem solving by Uninformed & Informed Search

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

**Performance Measure**:
- Minimize the total distance traveled by Rohan.
- Ensure all required locations are visited exactly once.
- Successful completion of the route from Admission Office *(start node)* to Exit *(end node)*.

**Environment**:
The university campus, represented as a graph with locations (nodes) and distances between them (edges).
- Static environment - distances between locations don't change during the execution.
- Fully observable - the agent has complete knowledge of the campus map and distances.
- Discrete - locations and distances are distinct and measurable.
- Deterministic - moving from one location to another always results in the same outcome (distance traveled).



**Actuators**:
Moving from one location (node) to an adjacent location.

**Sensors**:
- Knowledge of the current location.
- Access to the campus map (graph) and distances.
- Ability to sense whether a location has been visited before.

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [30]:
#Code Block : Set Initial State (Must handle dynamic inputs)
def get_user_input(graph):
    print("Available Locations:")
    for location in graph.keys():
        print(f"- {location}")
    start = input("Enter the start location: ").strip()
    end = input("Enter the end location: ").strip()
    return start, end

In [31]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
graph = {
    "Admission Office": {"Hostel Office": 2, "Library": 4},
    "Hostel Office": {"Admission Office": 2, "Library": 4, "Canteen": 6, "Hostel visit": 2},
    "Library": {"Admission Office": 4, "Hostel Office": 4, "Canteen": 7, "Dep’t visit": 3},
    "Canteen": {"Hostel Office": 6, "Library": 7, "Dep’t visit": 2, "Exit": 8, "Hostel visit": 6},
    "Dep’t visit": {"Library": 3, "Canteen": 2, "Exit": 5},
    "Hostel visit": {"Hostel Office": 2, "Canteen": 6, "Exit": 4},
    "Exit": {"Canteen": 8, "Dep’t visit": 5, "Hostel visit": 4}
}
all_nodes = set(graph.keys())





In [32]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def transition_function(path, neighbor):
  path.append(neighbor)
  return path

In [33]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are
def isGoal(currentNode, end_node, visited_nodes, graph):
  if (currentNode == end_node and set(visited_nodes) == all_nodes):
    return True
  return False

In [34]:
def heuristic(node, goal):
    """Heuristic function: Straight-line distance approximation"""
    min_edge_cost = min(min(edges.values()) for edges in graph.values() if edges)  # Smallest possible step
    return min_edge_cost * (len(graph) - len(node))

### 2.	Definition of Algorithm 1 (Iterative Deepening A* Algorithm)

In [35]:
import heapq

def ida_star(graph, start, goal, all_nodes):
    """Iterative Deepening A* (IDA*) algorithm to visit all nodes"""
    def search(path, g, bound, visited):
        node = path[-1]
        if isGoal(node, goal, visited, graph):
            return g, path
        f = g + heuristic(node, goal)
        if f > bound:
            return f, None
        min_bound = float('inf')
        for neighbor, cost in graph[node].items():
            if neighbor not in visited:  # Ensure all nodes are visited
                # path.append(neighbor)
                transition_function(path, neighbor)
                visited.add(neighbor)
                new_g = g + cost
                t, result = search(path, new_g, bound, visited)
                if result:
                    return t, result
                if t < min_bound:
                    min_bound = t
                path.pop()
                visited.remove(neighbor)
        return min_bound, None

    bound = heuristic(start, goal)
    path = [start]
    visited = {start}
    while True:
        t, result = search(path, 0, bound, visited)
        if result:
            return result, t
        if t == float('inf'):
            return None, float('inf')
        bound = t




### 3.	Definition of Algorithm 2 (Hill Climbing algorithm)

In [36]:
import random


# Compute the total distance of a given path
def path_distance(path, graph):
    """Calculate the total distance for a directed path."""
    total_distance = 0
    for i in range(len(path) - 1):
        if path[i+1] in graph[path[i]]:  # Check if the edge exists
            total_distance += graph[path[i]][path[i+1]]
        else:
            return float('inf')  # Invalid path (breaks direction)
    return total_distance

# Generate a random valid path from start to end
def random_path(start, end, all_nodes, graph):
    nodes = list(all_nodes - {start, end})
    random.shuffle(nodes)

    # Ensure a valid directed path by picking allowed edges
    path = [start]
    while path[-1] != end: #and isGoal(path[-1], end, path, graph):
        possible_next_nodes = list(set(graph[path[-1]].keys()) - set(path))  # Get valid moves
        if not possible_next_nodes:
            return None  # Dead end (invalid path)
        next_node = random.choice(possible_next_nodes)
        path.append(next_node)

    return path if isGoal(path[-1], end, path, graph) else None  # Ensure all nodes are visited

# Hill Climbing Algorithm
def hill_climb(graph, start, end, all_nodes):
    current_path = None
    while current_path is None:  # Ensure we get a valid starting path
        current_path = random_path(start, end, all_nodes, graph)

    current_distance = path_distance(current_path, graph) + heuristic(current_path[-1], end)

    while True:
        neighbors = []
        for i in range(1, len(current_path) - 2):  # Avoid swapping start/end
            for j in range(i + 1, len(current_path) - 1):
                new_path = current_path[:]
                new_path[i], new_path[j] = new_path[j], new_path[i]  # Swap

                # Ensure the new path is still valid and improves the heuristic
                new_distance = path_distance(new_path, graph) + heuristic(new_path[-1], end)
                if new_distance < float('inf'):
                    transition_function(neighbors, (new_path, new_distance))
                    # neighbors.append((new_path, new_distance))

        # Pick the best valid neighbor based on heuristic
        best_neighbor, best_distance = min(neighbors, key=lambda x: x[1], default=(None, None))
        if best_neighbor and best_distance < current_distance:
            current_path = best_neighbor
            current_distance = best_distance
        else:
            break  # Local minimum reached

    return current_path, path_distance(current_path, graph)

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [37]:
#Code Block : Function & call to get inputs (start/end state)

start_node, end_node = get_user_input(graph)

Available Locations:
- Admission Office
- Hostel Office
- Library
- Canteen
- Dep’t visit
- Hostel visit
- Exit
Enter the start location: Admission Office
Enter the end location: Exit


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [38]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

optimal_path, total_distance = ida_star(graph, start_node, end_node, all_nodes)

# Print results
if optimal_path:
    print("Optimal Path:", " → ".join(optimal_path))
    print("Total Distance:", total_distance)
else:
    print("No path found.")

Optimal Path: Admission Office → Library → Dep’t visit → Canteen → Hostel Office → Hostel visit → Exit
Total Distance: 21


In [39]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

best_path, best_distance = None, float('inf')
for _ in range(5):  # Try 5 random restarts
    path, dist = hill_climb(graph, start_node, end_node, all_nodes)
    # print("iteration ", _, path, dist)  # comment this line out at the end.
    if dist < best_distance:
        best_path, best_distance = path, dist

# Print the best result
print("Optimal Path:", " → ".join(best_path))
print("Total Distance:", best_distance)

Optimal Path: Admission Office → Library → Dep’t visit → Canteen → Hostel Office → Hostel visit → Exit
Total Distance: 21


### 5.	Comparitive Analysis (Time and Space Complexity)

In [40]:
#Code Block : Print the Time & Space complexity of algorithm 1
print("Iterative Deepening A* Search Complexity:")
print("Time Complexity:  O(b^d)  where b is the branching factor and d is the depth of the solution.") #In the worst case, IDA* may explore all possible paths up to the solution depth.
print("Space Complexity: O(bd) where b is the branching factor and d is the depth of the solution.") #IDA* performs a depth-first search, its space complexity is linear with the depth of the search.

Iterative Deepening A* Search Complexity:
Time Complexity:  O(b^d)  where b is the branching factor and d is the depth of the solution.
Space Complexity: O(bd) where b is the branching factor and d is the depth of the solution.


In [41]:
# Code Block : Print the Time & Space complexity of algorithm 2
print("\nHill Climbing Search Complexity:")
print("Time Complexity: O(number of iterations * cost of successor function)") #Depends on the heuristic and landscape
print("Space Complexity: O(b), where b is the average size of local neighbours/successors explored.") #Hill climbing only stores current state.


Hill Climbing Search Complexity:
Time Complexity: O(number of iterations * cost of successor function)
Space Complexity: O(b), where b is the average size of local neighbours/successors explored.


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

**Comparison** : Iterative Deepening A* guarantees an optimal solution (given an admissible heuristic), but can be slower. Hill Climbing is faster but may get stuck in local optima.
The choice of algorithm depends on whether optimality is crucial or a "good enough" solution is acceptable within a shorter time.